In [2]:
import pandas as pd
import numpy as np
from gym import spaces
import matplotlib.pyplot as plt
from scipy import stats
import random
from recsim import document
from recsim import user
from pathlib import Path
from user_sampler import LTSStaticUserSampler
from document_sampler import LTSDocumentSampler

from gym import Env
from gym.spaces import Discrete, Box
from numpy import int64

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from gym.spaces import Box, Discrete
from tqdm import tqdm

  
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

2023-02-20 18:57:38.567192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-20 18:57:38.720411: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/aayush/git/rl_recsys/.venv/lib/python3.9/site-packages/cv2/../../lib64:
2023-02-20 18:57:38.720455: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-20 18:57:39.605547: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.

In [2]:
sampler = LTSStaticUserSampler()
k=sampler.sample_user()
k.mode

/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:131: DeprecationWarning: This function is deprecated. Please call randint(20, 60 + 1) instead
  self._state_parameters["age"] = np.random.random_integers(20, 60)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:132: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  self._state_parameters["gender"] = np.random.random_integers(2)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:147: DeprecationWarning: This function is deprecated. Please call randint(1, 4 + 1) instead
  self._state_parameters["label"] = np.random.random_integers(4)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:149: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  self._state_parameters["key"] = np.random.random_integers(12)


1

In [3]:
df = pd.DataFrame(columns=['age', 'gender','valence','danceability','loudness','speechiness','acousticness','liveness','mood', 'tempo','key','energy','mode','duration_ms', 'instrumentalness'])
trained_users= pd.DataFrame(columns=['age', 'gender','valence','danceability','loudness','speechiness','acousticness','liveness','mood', 'tempo','key','energy','mode','duration_ms', 'instrumentalness'])
def getUser(User):
    sampler = LTSStaticUserSampler()
    k=sampler.sample_user()
    User=User.append({'age': k.age , 'gender':k.gender,'valence':k.valence,'danceability':k.danceability,'loudness':k.loudness,'speechiness':k.speechiness,'acousticness':k.acousticness,'liveness':k.liveness,'mood':k.label, "mode": k.mode,
                "key": k.key, "duration_ms": k.duration_ms,"tempo": k.tempo, "energy": k.energy, "instrumentalness": k.instrumentalness,}, ignore_index=True)
    return User

In [4]:
User=df.append(getUser(df))
trained_users=trained_users.append(User, ignore_index=True)


/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:131: DeprecationWarning: This function is deprecated. Please call randint(20, 60 + 1) instead
  self._state_parameters["age"] = np.random.random_integers(20, 60)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:132: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  self._state_parameters["gender"] = np.random.random_integers(2)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:147: DeprecationWarning: This function is deprecated. Please call randint(1, 4 + 1) instead
  self._state_parameters["label"] = np.random.random_integers(4)
/home/aayush/git/rl_recsys/src/recsim/user_sampler.py:149: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  self._state_parameters["key"] = np.random.random_integers(12)
/tmp/ipykernel_4054131/2097340565.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use 

In [5]:
trained_users

,age,gender,valence,danceability,loudness,speechiness,acousticness,liveness,mood,tempo,key,energy,mode,duration_ms,instrumentalness
0,44.0,1.0,0.990408,0.257983,0.773195,0.169707,0.858092,0.133329,3.0,210.191185,7.0,0.840216,1.0,238704.20238,0.181184


In [6]:
DATASET_NAME = "Spotify"
_DATA_PATH = Path(Path.home() / "rsys_data")
_DATASET_PATH = _DATA_PATH / DATASET_NAME
songs = pd.read_feather(_DATASET_PATH) 

In [7]:
songs.to_csv("spotify.csv")

In [11]:
songs_values=songs.drop(['name','artists','song_id'], axis=1)

In [ ]:
def discretize_column(self, column_name, num_bins):
        # Discretize a column by dividing it into num_bins equally-sized bins
        min_val = self.df[column_name].min()
        max_val = self.df[column_name].max()
        bin_size = (max_val - min_val) / num_bins
        bins = [min_val + i * bin_size for i in range(num_bins + 1)]
        return pd.cut(self.df[column_name], bins, labels=False, include_lowest=True)

In [17]:
import numpy as np
obs_low = np.zeros(len(songs_values)+2)
obs_high = np.ones(len(songs_values)+2)
observation_space = spaces.Box(low=obs_low, high=obs_high)
observation_space.sample()

/home/aayush/git/rl_recsys/.venv/lib/python3.9/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


array([0.8967746 , 0.43700942, 0.3650389 , ..., 0.91910666, 0.98444384,
       0.9500205 ], dtype=float32)

In [15]:
obs=spaces.Box(low=np.min(data), high=np.max(data), shape=data[0].shape, dtype=data.dtype)
obs.sample()

array([1769.67776253, 1405.9650874 , 1226.57990468,  267.47494684,
       1637.07194693, 1969.31098211,  619.22609301, 1966.98828995,
         86.13879871])

In [ ]:
song_dict=songs.to_dict()

In [ ]:
class SpotifyEnv(Env):
    def __init__(self):
        self.i=0
        self.action_space = Discrete(100)
        self.observation_space = Box(low=0, high=100000, shape=(1,10), dtype=int64)
        self.state = self.observation_space.sample().ravel()
        
    def step(self,action):
        self.i+=1 
        for j in range(self.i,10):
            self.state[j]=-1
        
        self.next_state = self.observation_space.sample().ravel()
        #self.next_state[0] = self.state[1]
        #self.next_state[1]= action
        
        if action in self.state:
            self.state[self.i]=action
            reward = -10
        else:  
            self.state[self.i]=action 
            reward = getReward1(ru, D, self.state, action)
        
        if self.i==9:
            print(self.state)
            done=True
        else:
            done=False
        info={}

        return self.state, reward, done, info
    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.i=0
        self.state = self.observation_space.sample().ravel()
        return self.state

In [33]:
import gym
from gym import spaces
import pandas as pd
import numpy as np

class SpotifyEnv(gym.Env):
    metadata = {'render.modes': ['human']}


    def __init__(self, df):
        super(SpotifyEnv, self).__init__()

        # Load the Spotify dataset from a CSV file
        self.df = songs

        # Get the audio features to use as the state space
        self.features = ['danceability', 'loudness',
                         'speechiness', 'acousticness', 'liveness',
                         'valence']

        # Set up the observation space
        obs_low = np.zeros(len(self.features)+2)
        obs_high = np.ones(len(self.features)+2)
        self.observation_space = spaces.Box(low=obs_low, high=obs_high)

        # Set up the action space
        self.action_space = spaces.Discrete(2)

    def get_observation(self):
        # Get the current observation by extracting the audio features, song name and year from the current track
        track_index = self.df.index
        audio_features = self.df.loc[track_index, self.features].values
        song_name = self.df.loc[track_index, 'name']
        song_year = self.df.loc[track_index, 'year']
        observation = np.concatenate((audio_features, [song_name, song_year]))
        return observation

In [34]:
env=SpotifyEnv(songs)

/home/aayush/git/rl_recsys/.venv/lib/python3.9/site-packages/gym/spaces/box.py:128: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [35]:
a=env.get_observation()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 6 and the array at index 1 has size 170653

In [24]:
a

array([0.78349143, 0.8573455 , 0.50605744, 0.24093565, 0.8928492 ,
       0.19299725, 0.54204285, 0.62446463], dtype=float32)

In [1]:
from p4 import UserState